# AutoKeras (0.4)
This notebook contains the experiments with AutoKeras, analyzing the quality and performance of the generated models and comparing them to handcrafted ones.

Main questions:
*   What does the structure of the generated networks look like?
*   How is the quality compared to handmade nets?

"Auto-Keras is an open source software library for automated machine learning (AutoML). It is developed by DATA Lab at Texas A&M University and community contributors. The ultimate goal of AutoML is to provide easily accessible deep learning tools to domain experts with limited data science or machine learning background. Auto-Keras provides functions to automatically search for architecture and hyperparameters of deep learning models." - *autokeras.com*

## Packages and Imports
The following section contains the packages that need to be installed and imported.

In google colab you have to **restart the environment** after installing the packages.

In [0]:
!pip install autokeras==0.4.0 # Version 0.4

In [0]:
!pip install torchviz # For model visualization

In [0]:
!python -V # Should output 3.6.* to work

In [0]:
import autokeras as ak
from autokeras.image.image_supervised import ImageClassifier
from autokeras.utils import pickle_from_file
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torchviz import make_dot
import torch
from google.colab import files

## Load data set
In the following sections you can load the data set you are going to use to test AutoKeras. The training data is contained in `x_train` and `y_train`, while the test data is in `x_test` and `y_test`.

### MNIST

In [0]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))
data_set_name = "mnist"

### Breast Cancer

In [0]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))
data_set_name = "breast_cancer"

## Create AutoKeras model

In [0]:
%tensorflow_version 1.x
clf = ImageClassifier(verbose=True, augment=True)
clf.fit(x_train, y_train, time_limit=1 * 60 * 60) # 1 Hour
clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)

### Export model

In [0]:
clf.export_autokeras_model(data_set_name + ".pkl")

In [0]:
# Download model
files.download(data_set_name + ".pkl")

### Load model

In [0]:
# Upload model
files.upload()

In [0]:
clf = pickle_from_file(data_set_name + ".pkl")

## Visualize and Evaluate
The visualization is still an ugly workaround, but it works.

In [0]:
# Evaluate model
print(clf.evaluate(x_test, y_test))

In [0]:
torch_model = clf.graph.produce_model() # For Portable models that got imported
#torch_model = clf.cnn.searcher.training_queue[-1][0].produce_model() # For models in the training queue

In [0]:
x = torch.randn([64, 1, 2]).requires_grad_(True)
y = torch_model(x)
make_dot(y, params=dict(list(torch_model.named_parameters()) + [('x', x)])).render(data_set_name, view=True)  

In [0]:
# Download visualization as PDF
files.download(data_set_name + ".pdf")

## References
https://autokeras.com/